<h3> Import the necessary packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from yahoo_historical import Fetcher

<h3> Read the Nasdaq (QQQ) index

In [2]:
ticker = 'QQQ'
start = [2019,1,1]

df = Fetcher(ticker, start).getHistorical()	
#df = df[['Date','Close']]
df.Date = pd.to_datetime(df.Date)

In [3]:
df.shape

(425, 7)

In [4]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,150.990005,155.750000,150.880005,154.880005,153.021332,58576700
1,2019-01-03,152.600006,153.259995,149.490005,149.820007,148.022064,74820200
2,2019-01-04,152.339996,157.000000,151.740005,156.229996,154.355118,74709300
3,2019-01-07,156.619995,158.860001,156.110001,158.089996,156.192795,52059300
4,2019-01-08,159.539993,160.110001,157.199997,159.520004,157.605637,49388700


<H3> create session gain/loss label for prediction

In [5]:
change = df.Close-df.Close.shift(1)
change.head()
df['gain_loss'] = change>0
df.gain_loss = df.gain_loss.astype(int)
df=df.iloc[1:,]
df.head(7)

,Date,Open,High,Low,Close,Adj Close,Volume,gain_loss
1,2019-01-03,152.600006,153.259995,149.490005,149.820007,148.022064,74820200,0
2,2019-01-04,152.339996,157.000000,151.740005,156.229996,154.355118,74709300,1
3,2019-01-07,156.619995,158.860001,156.110001,158.089996,156.192795,52059300,1
4,2019-01-08,159.539993,160.110001,157.199997,159.520004,157.605637,49388700,1
5,2019-01-09,160.139999,161.520004,159.470001,160.820007,158.890045,46491700,1
6,2019-01-10,159.600006,161.369995,158.699997,161.279999,159.344513,38943400,1
7,2019-01-11,160.330002,160.860001,159.789993,160.690002,158.761581,30176600,0


<h3> fundamentals

In [6]:
import pandas as pd
from searchtweets import load_credentials, gen_rule_payload, collect_results

In [7]:
all_tweets=[]

In [8]:
import os
import re
import pickle
rootdir = os.getcwd()

<h3> Read stock market news dataset

In [9]:
regex = re.compile('stock_market.*\.sav')
#'Run.*\.py$'

for root, dirs, files in os.walk(rootdir):
  for file in files:
    if regex.match(file):
        all_tweets.append(pickle.load(open(file,'rb')))
        print(file)
        

stock_market_2019-03-01-2019-03-30.sav
stock_market_2019-04-01-2019-04-29.sav
stock_market_2019-05-01-2019-05-31.sav
stock_market_2019-06-01-2019-06-28.sav
stock_market_2019-07-01-2019-07-31.sav
stock_market_2019-08-01-2019-08-30.sav
stock_market_2019-09-01-2019-09-30.sav
stock_market_2019-10-01-2019-10-31.sav
stock_market_2019-11-01-2019-11-22.sav
stock_market_2019-11-25-2019-12-30.sav


In [10]:
len(all_tweets)

10

In [11]:
#all_tweets[0]

In [12]:
text=[]
dates=[]
names=[]
for i in range(len(all_tweets)):
    text.append([tweet.all_text for tweet in all_tweets[i]])
    dates.append([tweet.created_at_datetime for tweet in all_tweets[i]])
    names.append([tweet.name for tweet in all_tweets[i]])

In [13]:

text1 = [item for sublist in text for item in sublist]
news = pd.DataFrame(text1, columns={'text'})
news['Date'] = [item for sublist in dates for item in sublist]
news['name'] = [item for sublist in names for item in sublist]

In [14]:
news.shape

(609, 3)

In [15]:
news

,text,Date,name
0,What the IPO market is telling us about the st...,2019-03-30 20:48:41,Yahoo Finance
1,S&amp;P 500 posts best three-month start to th...,2019-03-29 20:13:57,Yahoo Finance
2,The S&amp;P 500 is heading toward its best thr...,2019-03-29 13:48:19,Yahoo Finance
3,What the IPO market is telling us about the st...,2019-03-29 00:42:10,Yahoo Finance
4,What the IPO market is telling us about the st...,2019-03-28 17:25:15,Yahoo Finance
...,...,...,...
604,"S&amp;P 500, Dow, Nasdaq each clinch record cl...",2019-11-25 21:16:34,Yahoo Finance
605,Stock Market 2020: Goldman presents a gloomy ‘...,2019-11-25 16:35:26,Yahoo Finance
606,"S&amp;P 500, Nasdaq hit fresh record highs aft...",2019-11-25 15:29:48,Yahoo Finance
607,Stock futures advance after China IP announcem...,2019-11-25 13:33:21,Yahoo Finance


In [16]:
from datetime import timedelta

In [17]:
news.Date = pd.to_datetime(news.Date)
news.Date = [a-timedelta(hours=4) for a in news.Date]
news = news.sort_values('Date').reset_index(drop=True)
news.head()

,text,Date,name
0,S&amp;P 500 rises after three straight days of...,2019-03-01 15:45:21,Yahoo Finance
1,One of the things @BrianSozzi learned since 20...,2019-03-04 11:07:28,Yahoo Finance
2,Stock rise amid reports of impending U.S.-Chin...,2019-03-04 11:14:03,Yahoo Finance
3,What I learned about investing since the March...,2019-03-04 11:36:06,Yahoo Finance
4,Dow on track for worst day in 5 weeks &amp; Vi...,2019-03-04 13:31:36,Yahoo Finance


In [18]:
newsh=news.Date.astype('str')
#print(d)
newsh = [d[d.find(':')-2:d.find(':')] for d in newsh]
news['hour'] = newsh
news.hour = news.hour.astype(int)
news.head()

,text,Date,name,hour
0,S&amp;P 500 rises after three straight days of...,2019-03-01 15:45:21,Yahoo Finance,15
1,One of the things @BrianSozzi learned since 20...,2019-03-04 11:07:28,Yahoo Finance,11
2,Stock rise amid reports of impending U.S.-Chin...,2019-03-04 11:14:03,Yahoo Finance,11
3,What I learned about investing since the March...,2019-03-04 11:36:06,Yahoo Finance,11
4,Dow on track for worst day in 5 weeks &amp; Vi...,2019-03-04 13:31:36,Yahoo Finance,13


In [19]:
news.shape

(609, 4)

In [20]:
news[news.hour<=12].shape
news1 = news[news.hour<=12]

In [21]:
news1.dtypes

text            object
Date    datetime64[ns]
name            object
hour             int32
dtype: object

In [22]:
news1.shape

(354, 4)

In [23]:
import datetime as dt
news1.Date = pd.to_datetime(news1['Date']).dt.floor('d')
news1.head()

C:\Users\htadayyon\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,text,Date,name,hour
1,One of the things @BrianSozzi learned since 20...,2019-03-04,Yahoo Finance,11
2,Stock rise amid reports of impending U.S.-Chin...,2019-03-04,Yahoo Finance,11
3,What I learned about investing since the March...,2019-03-04,Yahoo Finance,11
7,Most investors have missed out on the stock ma...,2019-03-05,Yahoo Finance,4
8,Stocks slip at market open https://t.co/nbgNcB...,2019-03-05,Yahoo Finance,10


<h3> Merge stock market news dataset with stock price dataset

In [24]:
df = df.merge(news1, on = 'Date')

In [25]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,gain_loss,text,name,hour
0,2019-03-04,175.539993,175.789993,172.470001,174.419998,172.326843,38063500,1,One of the things @BrianSozzi learned since 20...,Yahoo Finance,11
1,2019-03-04,175.539993,175.789993,172.470001,174.419998,172.326843,38063500,1,Stock rise amid reports of impending U.S.-Chin...,Yahoo Finance,11
2,2019-03-04,175.539993,175.789993,172.470001,174.419998,172.326843,38063500,1,What I learned about investing since the March...,Yahoo Finance,11
3,2019-03-05,174.550003,175.089996,173.690002,174.550003,172.455292,22319900,1,Most investors have missed out on the stock ma...,Yahoo Finance,4
4,2019-03-05,174.550003,175.089996,173.690002,174.550003,172.455292,22319900,1,Stocks slip at market open https://t.co/nbgNcB...,Yahoo Finance,10


In [26]:
df.shape

(317, 11)

<h3> Remove URLs and usernames

In [27]:
drop_words = [list(filter(lambda word: word[0]!='@', text.split())) for text in df.text]
drop_words = [list(filter(lambda word: word[0:4]!='http', text)) for text in drop_words]
drop_words = [' '.join(a) for a in drop_words]

In [28]:
len(drop_words)

317

In [29]:
drop_words[0]

'One of the things learned since 2009 market bottom: Machine-driven trading will only amplify herd mentality: good news for traders and high-frequency platforms, bad news for long-term investors.'

In [30]:
df.text[0]

'One of the things @BrianSozzi learned since 2009 market bottom: Machine-driven trading will only amplify herd mentality: good news for traders and high-frequency platforms, bad news for long-term investors. @YahooFinance https://t.co/noTgt0XtBG'

In [31]:
df.text = drop_words

<h3> remove English stop words

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['this']
stopwords.extend(newStopWords)
example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords) 
  
word_tokens = word_tokenize(example_sent.lower()) 

filtered_sentence = [list(filter(lambda word: word not in stop_words, text.split())) for text in df.text]
filtered_sentence = [' '.join(a) for a in filtered_sentence]
filtered_sentence[0]

'One things learned since 2009 market bottom: Machine-driven trading amplify herd mentality: good news traders high-frequency platforms, bad news long-term investors.'

In [33]:
len(filtered_sentence)

317

In [34]:
df.text=filtered_sentence

In [35]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,gain_loss,text,name,hour
0,2019-03-04,175.539993,175.789993,172.470001,174.419998,172.326843,38063500,1,One things learned since 2009 market bottom: M...,Yahoo Finance,11
1,2019-03-04,175.539993,175.789993,172.470001,174.419998,172.326843,38063500,1,Stock rise amid reports impending U.S.-China t...,Yahoo Finance,11
2,2019-03-04,175.539993,175.789993,172.470001,174.419998,172.326843,38063500,1,What I learned investing since March 2009 stoc...,Yahoo Finance,11
3,2019-03-05,174.550003,175.089996,173.690002,174.550003,172.455292,22319900,1,Most investors missed stock market's epic 2019...,Yahoo Finance,4
4,2019-03-05,174.550003,175.089996,173.690002,174.550003,172.455292,22319900,1,Stocks slip market open via,Yahoo Finance,10


In [36]:
df.shape

(317, 11)

<h3> Create TFID Vector

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.text)

In [38]:
X_train_counts.shape

(317, 830)

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)    
X_train_tfidf.shape

(317, 830)

In [40]:
feature_names = count_vect.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=X_train_tfidf[0]
 
#print the scores
ft = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
ft=ft.sort_values(by=["tfidf"],ascending=False)
ft.head(10)

,tfidf
news,0.366478
platforms,0.223741
mentality,0.223741
long,0.223741
frequency,0.223741
herd,0.223741
machine,0.223741
amplify,0.223741
driven,0.223741
2009,0.208793


In [41]:
type(X_train_tfidf)

scipy.sparse.csr.csr_matrix

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2,2))
#X = vectorizer.fit_transform(df.text)
#print(vectorizer.get_feature_names())
processed_features = vectorizer.fit_transform(df.text).toarray()

In [72]:
print(X.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [73]:
from sklearn.model_selection import train_test_split
labels = df.gain_loss
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [74]:
X_train.shape

(253, 1632)

In [75]:
X_test.shape

(64, 1632)

In [76]:
X_train.min()

0.0

In [77]:
X_train.max()

1.0

<H3> Train classifier to predict stock price based on TFID vector

In [85]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [86]:
predictions = text_classifier.predict(X_test)

In [87]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[ 3 20]
 [ 2 39]]
              precision    recall  f1-score   support

           0       0.60      0.13      0.21        23
           1       0.66      0.95      0.78        41

    accuracy                           0.66        64
   macro avg       0.63      0.54      0.50        64
weighted avg       0.64      0.66      0.58        64

0.65625


In [88]:
y_test.value_counts()

1    41
0    23
Name: gain_loss, dtype: int64

In [89]:
predictions = text_classifier.predict(X_train)

In [90]:
y_train.value_counts()

1    167
0     86
Name: gain_loss, dtype: int64

In [91]:
print(confusion_matrix(y_train,predictions))
print(accuracy_score(y_train, predictions))

[[ 78   8]
 [  2 165]]
0.9604743083003953
